<a href="https://colab.research.google.com/github/stefanusaw/Prompt-Engineering-Fine-Tuning-LLMs/blob/main/PE_FTLLMs_UserStory2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
print("GPU Available:", torch.cuda.is_available())
print("GPU Count:", torch.cuda.device_count())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

GPU Available: True
GPU Count: 1
GPU Name: NVIDIA A100-SXM4-40GB


In [2]:
!pip install transformers datasets torch pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
import torch
torch.cuda.empty_cache()

dataset = load_dataset("ccdv/arxiv-summarization")

model_name = "google/pegasus-xsum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda")

def preprocess_data(examples):
    inputs = tokenizer(examples['article'], padding="max_length", truncation=True, max_length=512)
    outputs = tokenizer(examples['abstract'], padding="max_length", truncation=True, max_length=64)

    inputs['labels'] = outputs['input_ids']
    return inputs

small_train_dataset = dataset["train"].shuffle(seed=42).select(range(1000)).map(preprocess_data, batched=True)
small_val_dataset = dataset["validation"].select(range(100)).map(preprocess_data, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.96k [00:00<?, ?B/s]

train-00000-of-00015.parquet:   0%|          | 0.00/230M [00:00<?, ?B/s]

train-00001-of-00015.parquet:   0%|          | 0.00/228M [00:00<?, ?B/s]

train-00002-of-00015.parquet:   0%|          | 0.00/228M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [4]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [5]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=2,
    eval_strategy="epoch",
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_val_dataset,
)
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss
1,3.293900,2.929476
2,3.033300,2.886154
3,2.899300,2.879973


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 64, 'num_beams': 8, 'length_penalty': 0.6}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=3000, training_loss=3.1409603678385416, metrics={'train_runtime': 726.874, 'train_samples_per_second': 4.127, 'train_steps_per_second': 4.127, 'total_flos': 4334196621312000.0, 'train_loss': 3.1409603678385416, 'epoch': 3.0})

In [6]:
results = trainer.evaluate()
print(results)


{'eval_loss': 2.8799731731414795, 'eval_runtime': 4.9995, 'eval_samples_per_second': 20.002, 'eval_steps_per_second': 20.002, 'epoch': 3.0}


In [7]:
!pip install evaluate rouge_score


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.5 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=bb30c9ea0d9aba9bbb554ea36ec57138b77d6c3e1edba6ac8ecc4f1b53367790
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [13]:
from rouge_score import rouge_scorer
from tqdm import tqdm
import torch

# Initialize ROUGE scorer for ROUGE-1, ROUGE-2, and ROUGE-L
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

# Dictionary to store the ROUGE scores for each metric
rouge_scores = {
    "rouge1": [],
    "rouge2": [],
    "rougeL": []
}

# Function to generate summary using the fine-tuned model
def generate_summary(model, tokenizer, text, max_length=64, device='cuda'):
    # Move model to the same device as inputs
    model.to(device)

    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True, padding="max_length")

    # Move input tensors to the same device as the model
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Generate summary
    summary_ids = model.generate(inputs["input_ids"], max_length=max_length, num_beams=4, early_stopping=True)

    # Return decoded summary
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Set device to GPU if available, otherwise CPU
device = "cuda" if torch.cuda.is_available() else "cpu"

# Iterate over the validation dataset to generate summaries and calculate ROUGE scores
for example in tqdm(small_val_dataset):
    article = example["article"]
    reference_summary = example["abstract"]

    # Generate the model's summary for the article
    generated_summary = generate_summary(model, tokenizer, article, device=device)

    # Compute ROUGE scores
    scores = scorer.score(reference_summary, generated_summary)

    # Append results for each ROUGE metric
    rouge_scores["rouge1"].append(scores["rouge1"].fmeasure)
    rouge_scores["rouge2"].append(scores["rouge2"].fmeasure)
    rouge_scores["rougeL"].append(scores["rougeL"].fmeasure)

# Calculate average ROUGE scores
avg_rouge1 = sum(rouge_scores["rouge1"]) / len(rouge_scores["rouge1"])
avg_rouge2 = sum(rouge_scores["rouge2"]) / len(rouge_scores["rouge2"])
avg_rougeL = sum(rouge_scores["rougeL"]) / len(rouge_scores["rougeL"])

# Print average ROUGE scores
print(f"Average ROUGE-1: {avg_rouge1}")
print(f"Average ROUGE-2: {avg_rouge2}")
print(f"Average ROUGE-L: {avg_rougeL}")


100%|██████████| 100/100 [02:58<00:00,  1.79s/it]

Average ROUGE-1: 0.2633197077556673
Average ROUGE-2: 0.08391703355041477
Average ROUGE-L: 0.17333347543904426


In [14]:
model.save_pretrained("./summarization_model")
tokenizer.save_pretrained("./summarization_model")


('./summarization_model/tokenizer_config.json',
 './summarization_model/special_tokens_map.json',
 './summarization_model/spiece.model',
 './summarization_model/added_tokens.json',
 './summarization_model/tokenizer.json')

#Short Evaluation for Future Improvements
Due to the interest of computing capability and time, I decided to tackle User Story 2 using a smaller dataset, and also limiting numerous paremeters.

To name a few:


1. `inputs = tokenizer(examples['article'], padding="max_length", truncation=True, max_length=512)`
 --> limited to max_length of 512, but 1024 may fine-tune the LLM better
2.  `outputs = tokenizer(examples['abstract'], padding="max_length", truncation=True, max_length=64)
`--> similarly, max_length=64 instead of 256
3. `small_train_dataset = dataset["train"].shuffle(seed=42).select(range(1000)).map(preprocess_data, batched=True)`
and
`small_val_dataset = dataset["validation"].select(range(100)).map(preprocess_data, batched=True)`. --> trained on smaller dataset, but for better result, using the whole dataset is a possible future iteration improvement.
4. `training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=2,
    eval_strategy="epoch",
    fp16=True,
)` --> batch_size is just 1, and fp16=True is used to reduce the memory consumption. Increasing the batch_size will greatly improve the training result, and hence the Fine-tuned LLM will hold more quality.

